[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ourownstory/neural_prophet/blob/master/example_notebooks/autoregression_yosemite_temps.ipynb)

# Hyperparameter optimization with Ray Tune

We introduce the module for hyperparameter otpimization with Ray Tune.

It supports automatic tuning, with predefined by us hyperparameter sets, as well as manual tuning with user-provided configuration of the parameters.

Firstly, we will show how it works with NP model in automated mode.

In [ ]:
# install NeuralProphet from our repository
!pip install git+https://github.com/adasegroup/neural_prophet.git # may take a while
!pip install tensorboardX

In [1]:
import pandas as pd
import numpy as np
from neuralprophet import NeuralProphet

In [4]:
from neuralprophet.hyperparameter_tuner import tune_hyperparameters

In [5]:
if 'google.colab' in str(get_ipython()):
    data_location = "https://raw.githubusercontent.com/adasegroup/neural_prophet/master/"
else:
    data_location = "../"

df = pd.read_csv(data_location + "example_data/yosemite_temps.csv")
df.head(3)

,ds,y
0,2017-05-01 00:00:00,27.8
1,2017-05-01 00:05:00,27.0
2,2017-05-01 00:10:00,26.8


In [6]:
freq = '5min'
best_params, results_df = tune_hyperparameters('NP',
                                               df,
                                               freq)

RuntimeError: Unable to connect to Redis at 127.0.0.1:6379 after 12 retries. Check that 127.0.0.1:6379 is reachable from this machine. If it is not, your firewall may be blocking this port. If the problem is a flaky connection, try setting the environment variable `RAY_START_REDIS_WAIT_RETRIES` to increase the number of attempts to ping the Redis server.

This function by default outputs the dictionary of best hyperparameters chosen. It additionally will output the dataframe with detailed result of each trial if return_results is set to True.

In [ ]:
results_df[['config.growth', 'config.n_changepoints', 'config.changepoints_range',
       'config.trend_reg', 'config.yearly_seasonality',
       'config.weekly_seasonality', 'config.daily_seasonality',
       'config.seasonality_mode', 'config.seasonality_reg', 'config.n_lags',
       'config.d_hidden', 'config.num_hidden_layers', 'config.ar_sparsity',
       'config.learning_rate', 'config.loss_func', 'config.normalize']]

In [ ]:
best_params

This dictionary can further be used in initialization of NeuralProphet model.

This function has also additional parameters:
- **num_epochs**: Max possible number of epochs to train each model.
- **num_samples**: Number of samples from hyperparameter spaces to check.
- **resources_per_trial**: Resources per trial setting for ray.tune.run, {'cpu': 1, 'gpu': 2} for example



## Manual mode

In case of manual mode, a user must provide a config dictionary with hyperparameter spaces in compatability with Ray Tune api.

We provide a minimal example below, for more information on Search Spaces withit this link https://docs.ray.io/en/master/tune/api_docs/search_space.html?highlight=tune.choice

In [ ]:
from ray import tune

config = {'n_lags': tune.grid_search([10, 20, 30]),
          'learning_rate': tune.loguniform(1e-4, 1e-1),
          'num_hidden_layers': tune.choice([2, 8, 16])}

In [ ]:
freq = '5min'
best_params, results_df = tune_hyperparameters('NP', 
                                               df,
                                               freq, 
                                               mode = 'manual',
                                              config = config)

In [ ]:
results_df

In [ ]:
best_params